# Movie Recommendation System


This is a movie recommendation system project in which we will use content based
Recommendation method. 

In [1]:
#Importing Packages

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [2]:
#Here we importing the datasets
credits = pd.read_csv(r"C:\Users\dipak\OneDrive\Desktop\Movie Recommend\Datasets\tmdb_5000_credits.csv")
movies = pd.read_csv(r"C:\Users\dipak\OneDrive\Desktop\Movie Recommend\Datasets\tmdb_5000_movies.csv")

# Data Preprocessing


In [3]:
#first of all we merge the both dataset because workign on two different datasets it going to complicated for us

movies = movies.merge(credits,on = "title")

In [4]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
movies =movies[["id","title","genres","keywords","overview","popularity","cast"]]

In [12]:
#here we selecting only important columns and cheking for missing and duplicate values

movies.isnull().sum()
movies['overview']=movies['overview'].dropna()


C:\Users\dipak\AppData\Local\Temp\ipykernel_3580\2581393269.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['overview']=movies['overview'].dropna()


In [14]:
movies = movies.dropna()

In [16]:
#Here we converting the list dictionary into list

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    return L

movies["genres"] = movies["genres"].apply(convert)



In [17]:
movies["keywords"] = movies["keywords"].apply(convert)


In [18]:
#converting for cast 
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
             L.append(i["name"])
             counter+=1
        else:
            break
    return L
        


In [19]:
movies["cast"] = movies["cast"].apply(convert3)


In [20]:
movies.head()


,id,title,genres,keywords,overview,popularity,cast
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",150.437577,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",139.082615,"[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,107.376788,"[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,112.312950,"[Christian Bale, Michael Caine, Gary Oldman]"
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",43.926995,"[Taylor Kitsch, Lynn Collins, Samantha Morton]"


In [ ]:
#converting the crew column
'''
def fetch_director(obj):
        L = []
        for i in ast.literal_eval(obj):
            if i['job']=="Director":
                L.append(i["name"])
            else:
                break
        return L
'''

In [ ]:
#movies["crew"] = movies["crew"].apply(fetch_director)

In [21]:
movies["overview"] = movies["overview"].apply(lambda x:x.split())


In [23]:
# Now remowing from space from words. for eg dipak gaikwad = dipakgaiwjkwad

movies["overview"]=movies["overview"].apply(lambda x:[i.replace(" ","")for i in x])
movies["genres"]=movies["genres"].apply(lambda x:[i.replace(" ","")for i in x])
movies["keywords"]=movies["keywords"].apply(lambda x:[i.replace(" ","")for i in x])
movies["cast"]=movies["cast"].apply(lambda x:[i.replace(" ","")for i in x])

In [24]:
#merging five coluns in tag
movies['tag'] = movies['genres']+movies['keywords']+movies['overview']

In [25]:
movies.head()

,id,title,genres,keywords,overview,popularity,cast,tag
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[SamWorthington, ZoeSaldana, SigourneyWeaver]","[Action, Adventure, Fantasy, ScienceFiction, c..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",139.082615,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]","[Adventure, Fantasy, Action, ocean, drugabuse,..."
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",107.376788,"[DanielCraig, ChristophWaltz, LéaSeydoux]","[Action, Adventure, Crime, spy, basedonnovel, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",112.312950,"[ChristianBale, MichaelCaine, GaryOldman]","[Action, Crime, Drama, Thriller, dccomics, cri..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",43.926995,"[TaylorKitsch, LynnCollins, SamanthaMorton]","[Action, Adventure, ScienceFiction, basedonnov..."


In [26]:
#created new dataset (main)
main = movies[['id','title','tag','genres','popularity','cast']]

In [27]:
main.head()

,id,title,tag,genres,popularity,cast
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, c...","[Action, Adventure, Fantasy, ScienceFiction]",150.437577,"[SamWorthington, ZoeSaldana, SigourneyWeaver]"
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drugabuse,...","[Adventure, Fantasy, Action]",139.082615,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]"
2,206647,Spectre,"[Action, Adventure, Crime, spy, basedonnovel, ...","[Action, Adventure, Crime]",107.376788,"[DanielCraig, ChristophWaltz, LéaSeydoux]"
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dccomics, cri...","[Action, Crime, Drama, Thriller]",112.312950,"[ChristianBale, MichaelCaine, GaryOldman]"
4,49529,John Carter,"[Action, Adventure, ScienceFiction, basedonnov...","[Action, Adventure, ScienceFiction]",43.926995,"[TaylorKitsch, LynnCollins, SamanthaMorton]"


In [28]:
main['tag'] = main['tag'].apply(lambda x:" ".join(x))
main['genres'] = main['genres'].apply(lambda x:" ".join(x))
main['cast'] = main['cast'].apply(lambda x:" ".join(x))

C:\Users\dipak\AppData\Local\Temp\ipykernel_3580\3061806836.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main['tag'] = main['tag'].apply(lambda x:" ".join(x))


In [32]:
main.head()

,id,title,tag,genres,popularity,cast
0,19995,Avatar,Action Adventure Fantasy ScienceFiction cultur...,Action Adventure Fantasy ScienceFiction,150.437577,SamWorthington ZoeSaldana SigourneyWeaver
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action ocean drugabuse exoti...,Adventure Fantasy Action,139.082615,JohnnyDepp OrlandoBloom KeiraKnightley
2,206647,Spectre,Action Adventure Crime spy basedonnovel secret...,Action Adventure Crime,107.376788,DanielCraig ChristophWaltz LéaSeydoux
3,49026,The Dark Knight Rises,Action Crime Drama Thriller dccomics crimefigh...,Action Crime Drama Thriller,112.312950,ChristianBale MichaelCaine GaryOldman
4,49529,John Carter,Action Adventure ScienceFiction basedonnovel m...,Action Adventure ScienceFiction,43.926995,TaylorKitsch LynnCollins SamanthaMorton


In [35]:
# convert all String of tag into lower case : 
main['tag'] = main['tag'].apply(lambda x:x.lower())
main['genres'] = main['genres'].apply(lambda x:x.lower())
main['cast'] = main['cast'].apply(lambda x:x.lower())
main['title'] = main['title'].apply(lambda x:x.lower())

C:\Users\dipak\AppData\Local\Temp\ipykernel_3580\2707565089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main['tag'] = main['tag'].apply(lambda x:x.lower())
C:\Users\dipak\AppData\Local\Temp\ipykernel_3580\2707565089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main['genres'] = main['genres'].apply(lambda x:x.lower())
C:\Users\dipak\AppData\Local\Temp\ipykernel_3580\2707565089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [36]:
main.head()

,id,title,tag,genres,popularity,cast
0,19995,avatar,action adventure fantasy sciencefiction cultur...,action adventure fantasy sciencefiction,150.437577,samworthington zoesaldana sigourneyweaver
1,285,pirates of the caribbean: at world's end,adventure fantasy action ocean drugabuse exoti...,adventure fantasy action,139.082615,johnnydepp orlandobloom keiraknightley
2,206647,spectre,action adventure crime spy basedonnovel secret...,action adventure crime,107.376788,danielcraig christophwaltz léaseydoux
3,49026,the dark knight rises,action crime drama thriller dccomics crimefigh...,action crime drama thriller,112.312950,christianbale michaelcaine garyoldman
4,49529,john carter,action adventure sciencefiction basedonnovel m...,action adventure sciencefiction,43.926995,taylorkitsch lynncollins samanthamorton


In [37]:
#Stemming(going = go )
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

main['tag'] = main['tag'].apply(stem)

C:\Users\dipak\AppData\Local\Temp\ipykernel_3580\690289355.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main['tag'] = main['tag'].apply(stem)


Here Proprocessing part done

In [38]:
main.head()

,id,title,tag,genres,popularity,cast
0,19995,avatar,action adventur fantasi sciencefict culturecla...,action adventure fantasy sciencefiction,150.437577,samworthington zoesaldana sigourneyweaver
1,285,pirates of the caribbean: at world's end,adventur fantasi action ocean drugabus exotici...,adventure fantasy action,139.082615,johnnydepp orlandobloom keiraknightley
2,206647,spectre,action adventur crime spi basedonnovel secreta...,action adventure crime,107.376788,danielcraig christophwaltz léaseydoux
3,49026,the dark knight rises,action crime drama thriller dccomic crimefight...,action crime drama thriller,112.312950,christianbale michaelcaine garyoldman
4,49529,john carter,action adventur sciencefict basedonnovel mar m...,action adventure sciencefiction,43.926995,taylorkitsch lynncollins samanthamorton


In [39]:
#removing the unwanted words (stopwords)--and -- Vectorization 
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(main['tag']).toarray()

In [90]:
similarity = cosine_similarity(vectors)

def recommend(movie):
    if movie not in main['title'].values:
        print("Movie not found in the dataset")
        return

    movie_index = main[main['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[0:10]

    sorted_movies = []
    for i in movie_list:
        movie_title = main.iloc[i[0]]['title']
        popularity = main.iloc[i[0]]['popularity']
        sorted_movies.append((movie_title, popularity))

    sorted_movies = sorted(sorted_movies, key=lambda x: x[1], reverse=True)
    
    for movie, _ in sorted_movies:
        print(movie)

In [94]:

recommend('man of steel')

batman v superman: dawn of justice
man of steel
jupiter ascending
superman returns
superman ii
astro boy
superman iii
superman iv: the quest for peace
ong bak 2
x-men: first class
